In [19]:
import requests
import json
import mechanize
from bs4 import BeautifulSoup
import time
#initailize the browser
br = mechanize.Browser()
br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=10)
br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]

In [20]:
#获取所有比赛连接
start_link = 'http://data.huffingtonpost.com/2014/world-cup/matches/belgium-vs-usa-731822'
response = mechanize.urlopen(start_link)
soup = BeautifulSoup(response)
links_html = soup.find_all('span', class_='matchup')
links = []
for link_html in links_html:
    a = link_html.find_all('a')
    for l in a:
        link = l.get('href')
        link = link.split('/')[-1]
        links.append(link)
links[:5]

[u'brazil-vs-chile-731815',
 u'colombia-vs-uruguay-731816',
 u'netherlands-vs-mexico-731817',
 u'costa-rica-vs-greece-731818',
 u'france-vs-nigeria-731819']

In [17]:
#用json转成可读的结构
def get_match_data(match):
    match_id = match.split('-')[-1]
    response = mechanize.urlopen('http://data.huffingtonpost.com/2014/world-cup/matches/%s.json' % match_id)
    match_data = json.loads(response.read())
    return match_data
match_data = get_match_data(links[0])#test
match_data.keys()
match_data

{u'events': [{u'displaymin': 1,
   u'event_id': None,
   u'field_pass': u'f',
   u'id': None,
   u'min': 0,
   u'outcome': u'1',
   u'period': u'1',
   u'player_id': u'48852',
   u'qualifiers': {},
   u'sec': 0,
   u'side': u'A',
   u'team': u'831',
   u'to': u'122779',
   u'to_x': None,
   u'to_y': None,
   u'to_z': None,
   u'type': u'102',
   u'x': 0.0,
   u'y': 0.0},
  {u'displaymin': 1,
   u'event_id': None,
   u'field_pass': u'f',
   u'id': None,
   u'min': 0,
   u'outcome': u'1',
   u'period': u'1',
   u'player_id': u'122779',
   u'qualifiers': {},
   u'sec': 0,
   u'side': u'A',
   u'team': u'831',
   u'to': u'37269',
   u'to_x': None,
   u'to_y': None,
   u'to_z': None,
   u'type': u'102',
   u'x': 0.0,
   u'y': 0.0},
  {u'displaymin': 1,
   u'event_id': None,
   u'field_pass': u'f',
   u'id': None,
   u'min': 0,
   u'outcome': u'1',
   u'period': u'1',
   u'player_id': u'37269',
   u'qualifiers': {},
   u'sec': 0,
   u'side': u'A',
   u'team': u'831',
   u'to': u'48853',
   u

In [23]:
def get_match_names(match):
    response = mechanize.urlopen('http://data.huffingtonpost.com/2014/world-cup/matches/%s' % links[0]) #example page
    soup = BeautifulSoup(response)

    data = {}

    data_script = soup.findAll("script")[1] #gets the second script block. Hopefully all pages follow the same format
    data_lines = data_script.text.split('\n')

    for line in data_lines[1:]:
        try:
            #format of a variable is HPIN.variable = [list of dictionaries]
            #this tries to convert it to 
            line_data = line.split(' = ')
            name = line_data[0].split('.')[1]
            value = json.loads(line_data[1][:-1])
            data[name] = value
        except:
            print "Pass this line...", line #should only occur on blank lines - yeah, I know, lazy exception handling...
            
    return data

names = get_match_names(links[0])#test
#names.keys()
names

Pass this line... 


{u'awayTeam': 831,
 u'callbackPath': u'/2014/world-cup/matches/731815.json',
 u'homeTeam': 614,
 u'imageCallbackInterval': 180000,
 u'imageCallbackPath': u'/2014/world-cup/matches/731815/instagrams.json',
 u'players': [{u'birth_date': u'1981-08-27',
   u'birth_place': u'Cachoeiro do Itapemirim',
   u'caps': u'9',
   u'club': u'Paris Saint Germain',
   u'country': u'Brazil',
   u'first_name': u'Maxwell',
   u'first_nationality': u'Brazil',
   u'goals': u'0',
   u'height': u'176',
   u'id': 9809,
   u'jersey_num': u'14',
   u'join_date': u'2013-08-01',
   u'known_name': u'Maxwell',
   u'last_name': u'Cabelino Andrade',
   u'name': u'Maxwell',
   u'position': u'Defender',
   u'position_wc': None,
   u'preferred_foot': u'Left',
   u'real_position': u'Full Back',
   u'real_position_side': u'Left',
   u'short_name': u'Maxwell',
   u'team_id': 614,
   u'weight': u'73'},
  {u'birth_date': u'1983-05-06',
   u'birth_place': u'Juazeiro',
   u'caps': u'75',
   u'club': u'Barcelona',
   u'country':

In [24]:
#合并一下已经获取的数据
data = {}
for match in links:
    if match not in data:
        print match
        time.sleep(10)

        match_data = get_match_data(match)
        match_names = get_match_names(match)
        data[match] = {'data': match_data, 'names': match_names}

        print match, " done" 
    else:
        print match, " already processed"
print len(data.keys()) #make sure you have all 64 games
data['germany-vs-algeria-731820']

brazil-vs-chile-731815
Pass this line... 
brazil-vs-chile-731815  done
colombia-vs-uruguay-731816
Pass this line... 
colombia-vs-uruguay-731816  done
netherlands-vs-mexico-731817
Pass this line... 
netherlands-vs-mexico-731817  done
costa-rica-vs-greece-731818
Pass this line... 
costa-rica-vs-greece-731818  done
france-vs-nigeria-731819
Pass this line... 
france-vs-nigeria-731819  done
germany-vs-algeria-731820
Pass this line... 
germany-vs-algeria-731820  done
argentina-vs-switzerland-731821
Pass this line... 
argentina-vs-switzerland-731821  done
belgium-vs-usa-731822
Pass this line... 
belgium-vs-usa-731822  done
france-vs-germany-731824
Pass this line... 
france-vs-germany-731824  done
brazil-vs-colombia-731823
Pass this line... 
brazil-vs-colombia-731823  done
argentina-vs-belgium-731826
Pass this line... 
argentina-vs-belgium-731826  done
netherlands-vs-costa-rica-731825
Pass this line... 
netherlands-vs-costa-rica-731825  done
brazil-vs-germany-731827
Pass this line... 
brazil-v

HTTPError: HTTP Error 404: Not Found

In [25]:
import cPickle as pickle
#pickle.dump(data, open( "wc2014.p", "wb"))
pickle.dump(data[u'germany-vs-argentina-731830'], open('final.p', 'wb'))

In [13]:
data = pickle.load(open("wc2014.p", "rb"))
data[u'germany-vs-argentina-731830']
#有数据了
#获取决赛数据方便分析决赛
pickle.dump(data[u'germany-vs-argentina-731830'], open('final.p', 'wb'))

AttributeError: 'dict' object has no attribute 'encode'